# Import Statements #

In [1]:
import os
import topogenesis as tg
import pyvista as pv
import ipywidgets as widgets
import trimesh as tm
import scipy as sc
import numpy as np
import pandas as pd
from itertools import cycle
pv.set_jupyter_backend('trame')

## 01. Loading the base object ##

We are loading the base .obj file here to create voxels. The voxel size is considered here is 3x3x3m and the base object is extruded to 2m so that a grid of only 1 voxel in depth is created. In the Hub implementation we can consider adding code to create a 2d voxel grid with the current 3d implementation

In [2]:
vs = 3
unit = [vs, vs, vs]
tol = 1e-09
mesh_path = os.path.relpath(r"C:\Users\aditya.soman\source\repos\Loci_Python\Obj_Data\floorobject.obj")
mesh = tg.geometry.load_mesh(mesh_path)


In [3]:
sample_cloud, ray_origins = tg.geometry.mesh_sampling(mesh, unit, multi_core_process=False, return_ray_origin = True, tol=tol)

In [4]:
lattice = sample_cloud.voxelate(unit, closed=True)
print(type(lattice))
print(lattice.unit)
print(lattice.bounds)

<class 'topogenesis.datastructures.datastructures.lattice'>
[3 3 3]
[[  0   0  -3]
 [114 108   3]]


In [5]:
# initiating the plotter
p = pv.Plotter(notebook=True) # ITK plotter for interactivity within the python notebook (itkwidgets library is required)


# fast visualization of the point cloud
#sample_cloud.fast_notebook_vis(p)

# fast visualization of the lattice
lattice.fast_notebook_vis(p)

# adding the base mesh: light blue
mesh = pv.read(mesh_path)
#p.add_mesh(mesh, color='#abd8ff')

# adding the ray origins: dark blue
#p.add_points(pv.PolyData(ray_origins), color='#004887')

# plotting
p.show()


Widget(value='<iframe src="http://localhost:63992/index.html?ui=P_0x1a2eff419d0_0&reconnect=auto" class="pyvis…

## Create Various Performance matrices (Lattices in topogenesis terminology)
1. Quiteness : I have added a few points in Rhino as potential noise sources and the argmin distances from these points are considered as the level of quiteness for that voxel. Note quiteness is used and not dB levels since we are not calculating any sound insulations etc.
2. Distance from Facades : Distance from facades N,S,E,W. If the users have a priority to configure zones close to a specific facade 
3. Daylight : Simplified daylight analysis values for the voxels 
4. Distance from Lobby : It is the distance of the voxels from the Lobby entrances 
5. Distance from an Core: It is the distance of the voxels from the Core entrances 

In [17]:
# Csv to points 
Complete_file = pd.read_excel('Point_List.xlsx', sheet_name=0,engine='openpyxl',header = None )
design_criterias= list(Complete_file.head(n=0))
array_excel = Complete_file.to_numpy()
# Numpy array of points
Noise_sources = array_excel.T
# Create availability lattice
init_lattice = lattice +1
availability_lattice_voxels = tg.to_lattice(init_lattice, init_lattice)
voxel_coordinates= availability_lattice_voxels.centroids
flattened_lattice = lattice.flatten()

In [18]:
Eucledian_distance = sc.spatial.distance.cdist(Noise_sources,voxel_coordinates)
noise_from_each_source = Eucledian_distance.T
Average_quiteness_indexing= np.argmin(noise_from_each_source, axis=1)
Average_quiteness_values = []
for branch,index in zip(noise_from_each_source,Average_quiteness_indexing):
    Average_quiteness_values.append(branch[index])
Quiteeness_lattice_padded= np.array([num if boolean else 0 for boolean, num in zip(flattened_lattice, cycle(Average_quiteness_values))])
padded_array = np.array(Quiteeness_lattice_padded)
Quiteness_lattice_np = Quiteeness_lattice_padded.reshape(lattice.shape)
Quiteness_lattice =tg.to_lattice(Quiteness_lattice_np, Quiteness_lattice_np.shape)

Quiteness_lattice.shape

(39, 37, 3)

In [21]:
###### initiating the plotter
p = pv.Plotter(notebook=True)
# Create the spatial reference
grid = pv.ImageData()

# Set the grid envelope_lattice: shape because we want to inject our values
nx, ny, nz = lattice.shape
grid.dimensions = (nx + 1, ny + 1, nz + 1)  # Dimensions = cells + 1 for points
# The bottom left corner of the data set
grid.origin = lattice.minbound
# These are the cell sizes along each axis
grid.spacing = lattice.unit

# Add the data values to the cell data
grid.cell_data["Quiteness"] = Quiteness_lattice.flatten(order="F")  # Flatten the Lattice

# fast visualization of the lattice
lattice.fast_vis(p)

# adding the meshes

opacity = [0, 0.75, 0, 0.75, 1.0] 
clim = [0, 100]
p.add_volume(grid, cmap="magma", clim=clim,
             opacity=opacity, opacity_unit_distance=5,)
p.add_points( Noise_sources, color='#e63946')

p.show()

Widget(value='<iframe src="http://localhost:63992/index.html?ui=P_0x1a2947eef90_9&reconnect=auto" class="pyvis…